# PSG Complete Data Analysis - FootballDecoded

### Objetivo del Análisis

Este notebook realiza una **extracción exhaustiva y organizada** de todos los datos disponibles del **Paris Saint-Germain** para las temporadas **2023/24 y 2024/25**, utilizando el conjunto completo de wrappers desarrollados en FootballDecoded.

**Datos a extraer:**
- Estadísticas de equipo (temporada completa por competición)
- Estadísticas individuales (todos los jugadores de la plantilla)
- Eventos de partidos (goles, pases, disparos, etc.)
- Datos espaciales (coordenadas, mapas de calor, redes de pase)
- Métricas avanzadas (xG, xA, PPDA, build-up chains)
- Análisis específico del partido PSG vs Auxerre (01/09/2024)

**Fuentes de datos:**
- **FBref**: Estadísticas completas y eventos
- **Understat**: Métricas avanzadas (xGChain, PPDA, etc.)
- **WhoScored**: Datos espaciales y coordenadas

In [33]:
# Dependencias base
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Union
from datetime import datetime
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Sistema y paths
import sys
import os

# Determinar el directorio raíz del proyecto
current_dir = os.path.dirname(os.path.abspath(''))
project_root = os.path.dirname(os.path.dirname(current_dir))  # Desde analysis/tactical_analysis/ subir 2 niveles

# Añadir al path
sys.path.insert(0, project_root)

print(f"📁 Directorio del proyecto: {project_root}")
print(f"📁 Directorio actual: {current_dir}")
print(f"🔍 Python path añadido: {project_root}")

📁 Directorio del proyecto: /home/oriol/FD
📁 Directorio actual: /home/oriol/FD/Data/analysis
🔍 Python path añadido: /home/oriol/FD


In [34]:
# Método 1: Import directo desde wrappers (recomendado)
try:
    from wrappers import *
    print("✅ Wrappers importados correctamente usando 'from wrappers import *'")
    import_method = "star_import"
except ImportError as e:
    print(f"❌ Error con import *: {e}")
    
    # Método 2: Import específico
    try:
        import wrappers.fbref_data as fbref
        import wrappers.understat_data as understat
        import wrappers.whoscored_data as whoscored
        print("✅ Wrappers importados individualmente")
        import_method = "individual_import"
    except ImportError as e2:
        print(f"❌ Error con imports individuales: {e2}")
        
        # Método 3: Import desde scrappers (fallback)
        try:
            sys.path.append(os.path.join(project_root, 'scrappers'))
            from scrappers import FBref, Understat, WhoScored
            print("✅ Usando scrappers base directamente")
            import_method = "scrapper_import"
        except ImportError as e3:
            print(f"❌ Error crítico: {e3}")
            print("💡 Ejecuta: pip install -e . desde el directorio raíz")
            import_method = "failed"

# Verificar qué método funcionó
print(f"🔧 Método de import usado: {import_method}")

❌ Error con import *: No module named 'wrappers'
❌ Error con imports individuales: No module named 'wrappers'
❌ Error crítico: No module named 'scrappers'
💡 Ejecuta: pip install -e . desde el directorio raíz
🔧 Método de import usado: failed
